In [1]:
import torch
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
import numpy as np
import networkx as nx
import pickle
import os
import glob
from statistics import mode
import matplotlib.pyplot as plt

In [2]:
def preprocess_data(file_path, nrows):

    # Load the data from the file
    data = pd.read_csv(file_path, sep=',', nrows=nrows)

    data['arrival'] = pd.to_datetime(data['arrival'], errors='coerce')
    data['departure'] = pd.to_datetime(data['departure'], errors='coerce')

    data.rename(columns={'latitude': 'temp_latitude', 'longitude': 'latitude'}, inplace=True)
    data.rename(columns={'temp_latitude': 'longitude'}, inplace=True)

    return data

In [3]:
def create_graph_from_data(data, seconds_since_last_train, file_number = None):
    # Sort and filter data first
    filtered_data = data.sort_values(by=['train_number', 'train_unique_sequence'])

    # Create the graph
    G = nx.DiGraph()

    # Phase 1: Node Creation
    # Using both 'train_number' and 'train_unique_sequence' for a unique index
    filtered_data['node_id'] = filtered_data.apply(
        lambda row: f"{row['train_number']}_{row['train_unique_sequence']}", axis=1
    )

    # Select only specific columns for node attributes
    selected_attributes = ['train_number',
                           'train_unique_sequence',
                           'ocp_type', 
                           'freight',
                           'number_of_traction_units', 
                           'departure_delay_seconds',  
                           'arrival_delay_seconds', 
                           'longitude', 
                           'latitude', 
                           'sin_seconds_since_midnight_departure',
                           'cos_seconds_since_midnight_departure',
                           'sin_seconds_since_midnight_arrival',
                           'cos_seconds_since_midnight_arrival',
                           'day_value',
                           'trainpart_weight',
                           'primary_delay',
                           'operator_class', 
                           'category',
                           'primary_delay_label',
                           'nodes_to_classify',
                           'delay_jump'
                           ]
    

    
    node_attributes = filtered_data.set_index('node_id')[selected_attributes].to_dict('index')

    # Add 'is_supernode' attribute for regular nodes
    for node_id, attrs in node_attributes.items():
        attrs['file_number'] = file_number
        attrs['is_supernode'] = 0
    G.add_nodes_from(node_attributes.items())


    # Phase 2: Edge Creation
    for train_number, group in filtered_data.groupby('train_number'):
        node_ids = group['node_id'].tolist()
        #print(f"Train {train_number}: Node IDs: {node_ids}")
        for i in range(len(node_ids) - 1):

            if not G.has_edge(node_ids[i], node_ids[i + 1]):
                G.add_edge(node_ids[i], node_ids[i + 1])
            if not G.has_edge(node_ids[i + 1], node_ids[i]):
                G.add_edge(node_ids[i + 1], node_ids[i])

    # Phase 3: Create supernodes and connect them
    train_numbers = filtered_data['train_number'].unique()
    for train_number in train_numbers:
        supernode_id = f"supernode_{train_number}"

        # Get the train nodes with the same train number as the supernode
        train_nodes = filtered_data[filtered_data['train_number'] == train_number]
        
        # Calculate the maximum values for each attribute from the train nodes
        supernode_attributes = {}
        for attr in selected_attributes:
            if attr == 'train_number':
                supernode_attributes[attr] = train_number
            else:
                attr_values = train_nodes[attr].tolist()
                attr_values = [val for val in attr_values if val is not None]
                
                if attr_values:
                    if pd.api.types.is_numeric_dtype(attr_values):
                        supernode_attributes[attr] = max(attr_values)
                    elif attr in ['ocp_type', 'operator_class', 'category']:  # Categorical attributes
                        supernode_attributes[attr] = mode(attr_values)
                    else:
                        supernode_attributes[attr] = max(attr_values, key=lambda x: (isinstance(x, str), x))
                else:
                    supernode_attributes[attr] = np.nan
        
        supernode_attributes['file_number'] = file_number
        supernode_attributes['is_supernode'] = 1
        
        G.add_node(supernode_id, **supernode_attributes)
        
        train_nodes = filtered_data[filtered_data['train_number'] == train_number]['node_id']
        for node in train_nodes:
            if not G.has_edge(supernode_id, node):
                G.add_edge(supernode_id, node)  # Supernode to regular node
            if not G.has_edge(node, supernode_id):
                G.add_edge(node, supernode_id)


    # Phase 4: Adding edges between recently used stations
    recent_trains_at_station = {}
    supernode_connections = set()

    
    # Preprocess the data for time calculations
    data['time'] = pd.to_datetime(data['arrival']).fillna(pd.to_datetime(data['departure']))
    data['node_id'] = data.apply(lambda row: f"{row['train_number']}_{row['train_unique_sequence']}", axis=1)

    # Ensure the data is sorted by time
    data.sort_values(by='time', inplace=True)

    for db640_code, group in data.groupby('db640_code'):
        for i, row in group.iterrows():
            

            current_time = row['time']
            event_node = row['node_id']

            if db640_code in recent_trains_at_station:
                for train_time, train_number in recent_trains_at_station[db640_code]:
                    time_difference = abs((current_time - train_time).total_seconds())
                    
                    if time_difference <= seconds_since_last_train and train_number != row['train_number']:
                      
                        last_train_node = f"{train_number}_{group.loc[(group['db640_code'] == db640_code) & (group['train_number'] == train_number) & (group['time'] == train_time), 'train_unique_sequence'].iloc[0]}"
                     
                        if G.has_node(last_train_node) and G.has_node(event_node):
                          
                            if not G.has_edge(last_train_node, event_node):
                                G.add_edge(last_train_node, event_node)
                            if not G.has_edge(event_node, last_train_node):
                                G.add_edge(event_node, last_train_node)
                          
                        
                            # Track connection between supernodes
                            last_train_supernode = f"supernode_{train_number}"
                            event_train_supernode = f"supernode_{row['train_number']}"
                            supernode_connections.add((last_train_supernode, event_train_supernode))

     
            
            # Update recent trains list
            if db640_code not in recent_trains_at_station:
                recent_trains_at_station[db640_code] = []
            recent_trains_at_station[db640_code].append((current_time, row['train_number']))


    # Phase 5: Add Directed Edges Between Supernodes based on tracked connections
    for supernode1, supernode2 in supernode_connections:
        if G.has_node(supernode1) and G.has_node(supernode2):

            if not G.has_edge(supernode1, supernode2):
                G.add_edge(supernode1, supernode2)
            if not G.has_edge(supernode2, supernode1):
                G.add_edge(supernode2, supernode1)


    print("Information about graph:")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Is the graph directed: {G.is_directed()}")




    ############# This code can be uncommented to plot some graphs for testing
    """
    # Plot all nodes for testing
    # Find all subgraphs (connected components) in the graph
    subgraphs = [G.subgraph(c) for c in nx.strongly_connected_components(G)]

    # Filter subgraphs based on the number of nodes
    filtered_subgraphs = [subgraph for subgraph in subgraphs if 30 <= subgraph.number_of_nodes() <= 1000]

    # Print information about each filtered subgraph
    for i, subgraph in enumerate(filtered_subgraphs):
        print(f"Subgraph {i + 1}:")
        print(f"  Number of nodes: {subgraph.number_of_nodes()}")
        print(f"  Number of edges: {subgraph.number_of_edges()}")
        
        # Plot the subgraph
        pos = nx.spring_layout(subgraph, seed=42)  # Use a fixed seed for reproducibility
        
        # Create a new figure with a larger size
        fig, ax = plt.subplots(figsize=(20, 20))
        
        # Draw nodes with smaller size
        nx.draw_networkx_nodes(subgraph, pos, node_size=300, ax=ax)
        
        # Draw edges with arrows
        nx.draw_networkx_edges(subgraph, pos, arrows=True, arrowsize=20, ax=ax)
        
        # Draw node labels
        node_labels = {node: str(node) for node in subgraph.nodes()}
        nx.draw_networkx_labels(subgraph, pos, labels=node_labels, font_size=12, font_weight='bold', ax=ax)
        
        # Draw node ID labels
        node_id_labels = nx.get_node_attributes(subgraph, 'node_id')
        node_id_pos = {node: (x, y - 0.05) for node, (x, y) in pos.items()}  # Adjust the position of node ID labels
        nx.draw_networkx_labels(subgraph, node_id_pos, labels=node_id_labels, font_size=10, font_color='red', ax=ax)
        
        # Remove axis
        ax.axis('off')
        
        # Create the 'plots' folder if it doesn't exist
        if not os.path.exists('plots'):
            os.makedirs('plots')
        
        # Save the subgraph plot to the 'plots' folder
        plt.tight_layout()
        plt.savefig(f'plots/subgraph_{file_number}_{i + 1}.png', dpi=300)
        plt.close(fig)  # Close the figure to free up memory

    # Print the number of filtered subgraphs
    print(f"Number of subgraphs with 30 to 1000 nodes: {len(filtered_subgraphs)}")

    """  

    return G

In [4]:
def extract_node_features_and_labels(G):
    x_list = []
    y_list = []

    nodes_to_train_on_count = 0
    nodes_not_to_train_on_count = 0

    train_label_one_count = 0
    train_label_zero_count = 0

    for node in G.nodes:

        # Example node attribute extraction
        node_attributes = G.nodes[node]

        # Extract input features
        train_number = node_attributes['train_number']
        train_unique_sequence = node_attributes['train_unique_sequence']
        ocp_type = node_attributes['ocp_type']
        freight = node_attributes['freight']
        number_of_traction_units = node_attributes['number_of_traction_units']
        departure_delay_seconds = node_attributes['departure_delay_seconds']
        arrival_delay_seconds = node_attributes['arrival_delay_seconds']
        longitude = node_attributes['longitude']
        latitude = node_attributes['latitude']
        sin_departure = node_attributes['sin_seconds_since_midnight_departure']
        cos_departure = node_attributes['cos_seconds_since_midnight_departure']
        sin_arrival = node_attributes['sin_seconds_since_midnight_arrival']
        cos_arrival = node_attributes['cos_seconds_since_midnight_arrival']
        day_value = node_attributes['day_value']
        trainpart_weight = node_attributes['trainpart_weight']
        operator_class = node_attributes['operator_class']
        category = node_attributes['category']
        primary_delay_label = node_attributes['primary_delay_label']
        primary_delay = node_attributes['primary_delay']
        delay_change = node_attributes['delay_jump']

        nodes_to_classify = node_attributes['nodes_to_classify']
        file_number = node_attributes['file_number']
        is_supernode = node_attributes['is_supernode']

        # Encode binary and categorical features
        encoded_nodes_to_classify = 1 if nodes_to_classify else 0
        encoded_primary_delay_label = 1 if primary_delay_label else 0
        encoded_ocp_type = 1 if ocp_type == 'Stop' else 0
        encoded_freight = 1 if freight else 0

        # We only want to train on nodes that are not supernodes
        nodes_to_train_on = encoded_nodes_to_classify and not is_supernode

        if nodes_to_train_on:
            nodes_to_train_on_count += 1
            if encoded_primary_delay_label == 1:
                train_label_one_count += 1
            else:
                train_label_zero_count += 1
        else:
            nodes_not_to_train_on_count += 1


        # Append input features to the list
        x_list.append([
            category, operator_class, longitude, latitude, trainpart_weight,
            departure_delay_seconds, arrival_delay_seconds, delay_change, train_unique_sequence,
            number_of_traction_units, sin_departure, cos_departure,
            sin_arrival, cos_arrival, day_value, encoded_freight, 
            encoded_ocp_type, nodes_to_train_on, 
            train_number, file_number, is_supernode
        ])

        y_list.append((encoded_primary_delay_label, float(primary_delay)))


    x = torch.tensor(x_list, dtype=torch.float)
    y = torch.tensor(y_list, dtype=torch.float)


    # Check for NaN values
    if np.isnan(x.numpy()).any() or np.isnan(y.numpy()).any():
        raise ValueError("NaN values detected in x or y tensors.")
    
    print(f"Nodes to train on: {nodes_to_train_on_count}")
    print(f"Nodes not to train on: {nodes_not_to_train_on_count}")
    print(f"Nodes to train on with label 1: {train_label_one_count}")
    print(f"Nodes to train on with label 0: {train_label_zero_count}")


    return x, y

In [5]:
def create_pyg_data_object(G, x, y):
    node_to_int = {node: i for i, node in enumerate(G.nodes)}
    edges_as_int = [(node_to_int[src], node_to_int[dst]) for src, dst in G.edges]
    edge_index = torch.tensor(edges_as_int).t().contiguous()

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

In [6]:
def process_graph(G, min_nodes, set_assignment):
    undirected_G = G.to_undirected()
    components = list(nx.connected_components(undirected_G))
    data_objects = []
    max_nodes = 0

    for j, component in enumerate(components):
        component_size = len(component)
        if component_size > min_nodes:
            subgraph = G.subgraph(component)
            print(f"Information about SUBGRAPH: {j}")
            print(f"Number of nodes: {subgraph.number_of_nodes()}")
            print(f"Number of edges: {subgraph.number_of_edges()}")
            print(f"Is the graph directed: {subgraph.is_directed()}")


            max_nodes = max(max_nodes, subgraph.number_of_nodes())
            x, y = extract_node_features_and_labels(subgraph)
            pyg_data_object = create_pyg_data_object(subgraph, x, y)

            pyg_data_object.set_type = set_assignment 

            data_objects.append(pyg_data_object)

    print(f"Largest graph size: {max_nodes}")
    return data_objects


In [7]:
def process_files(file_paths, nrows, min_nodes, seconds_since_last_train, output_file_path):
    all_data_objects = []
    i = 0
    num_files = len(file_paths)

    # Adjusting for equal distribution of augmentation methods in the train set
    num_train_files = int(0.8 * num_files)  # 80% of files are for training
    #augmentation_split = num_train_files // 2 # Divide train files equally among the 3 methods

    num_test_files = int(0.10 * num_files) + num_train_files  # Next 10% after train files
    # Remaining files are for validation


    for file_path in file_paths:
        data = preprocess_data(file_path, nrows)
        i += 1

        file_type = 0 if i <= num_train_files else (1 if i <= num_test_files else 2)

        print("We now have a file of type:")
        print(file_type)

        # Keep track of the processing method
        print(f"Processing file {i}")

        # Adjusting processing based on file index
        if i <= num_files:  

            print(f"Processing file {i}")
            G = create_graph_from_data(data, seconds_since_last_train=seconds_since_last_train, file_number=i)
            all_data_objects.extend(process_graph(G, min_nodes, file_type))


    with open(output_file_path, 'wb') as file:
        pickle.dump(all_data_objects, file)

   
    return all_data_objects

In [8]:
# Specify the directory where the CSV files are located
data_directory = 'data'  # Replace with your directory path

# List all CSV files in the directory
file_paths = glob.glob(os.path.join(data_directory, '*.csv'))
nrows = None  # Set to None to read all rows or specify a number to read a subset
min_nodes = 1000 # Minimum number of nodes in a subgraph to be considered
seconds_since_last_train = 180
output_file_path = 'dataset.pkl'

all_data_objects = process_files(file_paths, nrows, min_nodes, seconds_since_last_train, output_file_path)


We now have a file of type:
2
Processing file 1
Processing file 1
Information about graph:
Number of nodes: 289123
Number of edges: 1387728
Is the graph directed: True
Information about SUBGRAPH: 0
Number of nodes: 280064
Number of edges: 1354700
Is the graph directed: True
Nodes to train on: 8795
Nodes not to train on: 271269
Nodes to train on with label 1: 0
Nodes to train on with label 0: 8795
Information about SUBGRAPH: 1
Number of nodes: 34
Number of edges: 132
Is the graph directed: True
Nodes to train on: 2
Nodes not to train on: 32
Nodes to train on with label 1: 0
Nodes to train on with label 0: 2
Information about SUBGRAPH: 2
Number of nodes: 23
Number of edges: 86
Is the graph directed: True
Nodes to train on: 1
Nodes not to train on: 22
Nodes to train on with label 1: 0
Nodes to train on with label 0: 1
Information about SUBGRAPH: 3
Number of nodes: 5
Number of edges: 14
Is the graph directed: True
Nodes to train on: 1
Nodes not to train on: 4
Nodes to train on with label 1